In [13]:

import os 
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '1Ut7cvQn4_Czj-uYeekhm3QDMO5EBVehhkefGPZaFpXM'
def main():
    credentials = None

    if os.path.exists('token.json'):
        credentials = Credentials.from_authorized_user_file('token.json', SCOPES)

    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            credentials = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(credentials.to_json())

    try:
        service = build('sheets', 'v4', credentials=credentials)
        sheets = service.spreadsheets()
        result = sheets.values().get(spreadsheetId=SPREADSHEET_ID, range='feed!A2:A3').execute()

        values = result.get('values', [])
        links = []  

        for row in values:
            links.append(row)  

        return links 

    except HttpError as error:
        print(error)

if __name__ == '__main__':
    links = main()
    print(type(links[0]))


<class 'list'>


In [12]:
import pandas as pd
import requests

def get_image_size(url):
    try:
        response = requests.head(url)  
        if response.status_code == 200:  
            content_length = response.headers.get('content-length')  
            if content_length:
                size_in_bytes = int(content_length)
                size_in_kb = size_in_bytes / 1024  
                return size_in_kb
        else:
            print(f"Error: {response.status_code}")  
        print(f"An error occurred: {e}")
        return None

def create_dataframe(links):
    data = []
    for link in links:
        size = get_image_size(link)
        if size:
            data.append({'Посилання': link, 'Розмір (KB)': size})

    df = pd.DataFrame(data)
    return df

if __name__ == '__main__':

    dataframe = create_dataframe(links)
    print(dataframe)



An error occurred: No connection adapters were found for "['https://data.sanitino.eu/PRODUCT-33916/8607663ccebd28fe87b63f8f?size=feed-1080']"
An error occurred: No connection adapters were found for "['https://data.sanitino.eu/PRODUCT-62434/f7aa3c2884a870a185e9232b?size=feed-1080']"
Empty DataFrame
Columns: []
Index: []
